In [1]:
# import libraries
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import sys
import inspect
import time
import pickle

# access parent directory from notebooks directory
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)

import src.simulation as s
import src.utils as utils

%load_ext autoreload
%autoreload 2
%matplotlib inline
%reload_ext autoreload

# Online-Communication Network Data


In [2]:
# instantiate simulation class
simulation =  s.Simulation()

/Users/davidblok/Work EUR/GitHub/ABM-PA/src/simulation.py:24: DtypeWarning: Columns (319,321,323,377,434,436,439,441,444,446,449,451,454,456,459,461,464,466,469,471,474,476,479,481,484,486,489,491) have mixed types.Specify dtype option on import or set low_memory=False.
  self.nominationPopulation = p.PeerNominatedDataPopulation('Peer-Nomination data population', self.input_args)
/Users/davidblok/.pyenv/versions/3.7.0/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (22) have mixed types.Specify dtype option on import or set low_memory=False.
  
/Users/davidblok/Work EUR/GitHub/ABM-PA/src/simulation.py:25: DtypeWarning: Columns (319,321,323,377,434,436,439,441,444,446,449,451,454,456,459,461,464,466,469,471,474,476,479,481,484,486,489,491) have mixed types.Specify dtype option on import or set low_memory=False.
  self.communicationPopulation = p.CommunicationDataPopulation('Communication data population', self.input_args)
/Users/davidblok/Work EUR/GitHub/ABM-P

In [3]:
# Read tuned parameter combinations (See tuning_com_network.ipynb)
pars_communication = pd.read_csv('../output/opt_pars_m_communication_revised.csv', sep=',', header=0, encoding='latin-1')
pars_communication

,threshold,ipa,error
0,0.0100,0.0025,0.000003
1,0.0075,0.0050,0.000152
2,0.0100,0.0050,0.000160
3,0.0125,0.0025,0.000489
4,0.0025,0.0175,0.000540
...,...,...,...
95,0.0500,0.0050,0.015171
96,0.0325,0.0150,0.015241
97,0.0475,0.0100,0.015369
98,0.0425,0.0125,0.015477


## 1. Pre-Intervention simulations

In [4]:
# Run pre-intervention for each parameter combination
list_pre_results_child_com = []
list_pre_results_class_com = []
list_snapshot_pa_com = []

for index, row in pars_communication.iterrows():
    start = time.time()
    # run pre-intervention
    pre_results_com, pre_results_avg_com, snapshot_pa_com = simulation.simulate_preinterventions(730,'communication',row['threshold'],row['ipa'])

    list_pre_results_child_com.append(pre_results_com)
    list_pre_results_class_com.append(pre_results_avg_com)
    list_snapshot_pa_com.append(snapshot_pa_com)

    end = time.time()
    print(index, row['threshold'], row['ipa'], "Time elapsed:", end - start, ' seconds')

# Save model output to object
file = open('../output/simulation/communication/list_pre_results_child.obj', 'wb')
pickle.dump(list_pre_results_child_com, file)

file = open('../output/simulation/communication/list_pre_results_class.obj', 'wb')
pickle.dump(list_pre_results_class_com, file)

file = open('../output/simulation/communication/list_snapshot_pa.obj', 'wb')
pickle.dump(list_snapshot_pa_com, file)


0 0.01 0.0025 Time elapsed: 7.725619077682495  seconds
1 0.0075 0.005 Time elapsed: 9.170293092727661  seconds
2 0.01 0.005 Time elapsed: 9.170247793197632  seconds
3 0.0125 0.0025 Time elapsed: 7.651893854141235  seconds
4 0.0025 0.017499999999999998 Time elapsed: 7.545680999755859  seconds
5 0.0075 0.0025 Time elapsed: 7.3930418491363525  seconds
6 0.0125 0.005 Time elapsed: 7.305781126022339  seconds
7 0.005 0.005 Time elapsed: 7.185580015182495  seconds
8 0.015 0.0025 Time elapsed: 7.108610153198242  seconds
9 0.01 0.0075 Time elapsed: 6.999501943588257  seconds
10 0.0125 0.0075 Time elapsed: 6.983995199203491  seconds
11 0.005 0.035 Time elapsed: 7.073143005371094  seconds
12 0.015 0.005 Time elapsed: 7.068781852722168  seconds
13 0.005 0.0025 Time elapsed: 7.055122137069702  seconds
14 0.017499999999999998 0.0025 Time elapsed: 6.9879310131073  seconds
15 0.005 0.0325 Time elapsed: 7.100682020187378  seconds
16 0.0125 0.01 Time elapsed: 6.995992660522461  seconds
17 0.0075 0.0475 

## 2. Interventions simulation

In [5]:
effect = np.array([0.05, 0.1, 0.15, 0.17, 0.2])

for effect in effect:
    # Run interventions for each parameter combination
    list_results_child_com = []
    list_results_class_com = []
    list_influential_agents_com = []

    for index, row in pars_communication.iterrows():
        start = time.time()
        # run intervention
        results_com, results_avg_com, agents_per_intervention_com = simulation.simulate_interventions(730,'communication', list_snapshot_pa_com[index], row['threshold'], row['ipa'], effect, index)


        list_results_child_com.append(results_com)
        list_results_class_com.append(results_avg_com)
        list_influential_agents_com.append(agents_per_intervention_com)

        end = time.time()
        print(effect, index, row['threshold'], row['ipa'], "Time elapsed:", end - start, ' seconds')

    # Save model output to object
    name1 = ('../output/simulation/communication/list_results_child_'+str(effect)+'.obj')
    file = open(name1, 'wb')
    pickle.dump(list_results_child_com, file)

    name2 = ('../output/simulation/communication/list_results_class_'+str(effect)+'.obj')
    file = open(name2, 'wb')
    pickle.dump(list_results_class_com, file)

    name3 = ('../output/simulation/communication/list_influential_agents_'+str(effect)+'.obj')
    file = open(name3, 'wb')
    pickle.dump(list_influential_agents_com, file)

0.05 0 0.01 0.0025 Time elapsed: 42.603989124298096  seconds
0.05 1 0.0075 0.005 Time elapsed: 42.61818194389343  seconds
0.05 2 0.01 0.005 Time elapsed: 42.41531682014465  seconds
0.05 3 0.0125 0.0025 Time elapsed: 42.34088611602783  seconds
0.05 4 0.0025 0.017499999999999998 Time elapsed: 43.54031300544739  seconds
0.05 5 0.0075 0.0025 Time elapsed: 42.08029007911682  seconds
0.05 6 0.0125 0.005 Time elapsed: 42.11077308654785  seconds
0.05 7 0.005 0.005 Time elapsed: 42.279759883880615  seconds
0.05 8 0.015 0.0025 Time elapsed: 48.10534620285034  seconds
0.05 9 0.01 0.0075 Time elapsed: 44.74602007865906  seconds
0.05 10 0.0125 0.0075 Time elapsed: 44.01580286026001  seconds
0.05 11 0.005 0.035 Time elapsed: 46.32616710662842  seconds
0.05 12 0.015 0.005 Time elapsed: 43.98815202713013  seconds
0.05 13 0.005 0.0025 Time elapsed: 43.784745931625366  seconds
0.05 14 0.017499999999999998 0.0025 Time elapsed: 43.83305597305298  seconds
0.05 15 0.005 0.0325 Time elapsed: 46.3775680065155